In [ ]:
import httplib2
from bs4 import BeautifulSoup, SoupStrainer
import requests
import os
import platform
import time
import json
from PyPDF2 import PdfFileReader
import io

import numpy as np
import pandas as pd
import re
import array
from time import sleep

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

In [ ]:
# click privacy policy button, pass in webdriver and data safety urls with privacy policy button on page
def press_button(driver, data_url):
    driver.implicitly_wait(1)
    driver.get(data_url)
    print(data_url)
    # press privacy policy button by LINK_TEXT from html/web inspection page
    
    try:
        button = driver.find_element(By.LINK_TEXT, "privacy policy")
        button.click()
    
    except NoSuchElementException:
        pass
        current = "N/A"
        print(current)
        return current
    
    
    # close unused tabs, url for privacy policy is child window
#     p = driver.window_handles[0]
#     c = driver.window_handles[1]
#     driver.switch_to.window(c)
    
#     WebDriverWait(driver, 2)
    
#     current = driver.current_url
#     print(current)
    comp = driver.current_url
    tabs_open = len(driver.window_handles)

    if tabs_open > 1:        
        p = driver.window_handles[0]
        c = driver.window_handles[1]
        driver.switch_to.window(c)

        WebDriverWait(driver, 4)

        current = driver.current_url
        print(current)
        # close child window and switch to original/parent window
        driver.close()
    
        driver.switch_to.window(p)
    else:
        WebDriverWait(driver, 2)
        current = driver.current_url
        print(current)

    sleep(1)

    # return privacy policy url for df
    return current

In [ ]:
# scroll search page to get all returned app url's and id's
def scroll(driver, timeout):
    last_height = driver.execute_script("return document.body.scrollHeight")
    scroll_pause_time = timeout
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(scroll_pause_time)
        
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
            
        last_height = new_height

In [ ]:
# get urls from search
def get_urls(starts_with):
    http = httplib2.Http()
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--incognito")
    if platform.system() == 'Windows':
        driver = webdriver.Chrome(executable_path='./chromedriver.exe')
    else:
        if platform.system() == 'Darwin':
            driver = webdriver.Chrome()
    # wait for page to load
    driver.implicitly_wait(30)
    # provide url(s) for driver
    driver.get("https://play.google.com/store/search?q=" + starts_with  + "&c=apps")
    # scroll to get all apps
    scroll(driver, 2)

    # read and obtain all urls
    if platform.system() == 'Windows':
        soup_a = BeautifulSoup(driver.page_source, features='html.parser')
    else:
        if platform.system() == 'Darwin':
            soup_a = BeautifulSoup(driver.page_source, 'lxml')
    
    driver.close()

    # copy all urls
    urls = []
    for link in soup_a.find_all('a'):
        urls.append(link.get('href'))
    return urls

In [ ]:
# alphabet = "abcdefghijklmnopqrstuvwxyz"
# categories = ["Daydream", "Beauty", "Auto & Vehicles", "Business", "Comics", "Books & Reference", "Dating", "Education", "Communication", "Food & Drink", "Entertainment", "Health & Fitness", "Events", "Finance", "Libraries & Demo", "House & Home", "Lifestyle", "Medical", "Maps & Navigation", "News & Magazines", "Audio", "Personalization", "Photography", "Parenting", "Shopping", "Productivity", "Tools", "Social", "Sports", "Video Players & Editors", "Travel & Local", "Weather",
#               "Adventure", "Arcade", "Action", "Board", "Casino", "Casual", "Card", "Music", "Puzzle", "Educational", "Role Playing", "Racing", "Sports", "Strategy", "Simulation", "Brain Games", "Trivia", "Word", "Education", "Creativity", "Video", "TV", "Weather", "a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z"]
# categories = ['word']
# urls = []
# alphabet = 'ab'

# loop through search categories and add to urls array
# for char_index in range(len(categories)):
#     print(categories[char_index])
#     urls += get_urls(categories[char_index])
#     print(urls[char_index])
# print(len(urls))
# for i in range(len(urls)):
#     print(urls[i])


In [ ]:
def get_data_safety_urls(app_ids):
    
    base_url = "https://play.google.com/store/apps/datasafety?id="
    data_urls = []
    
    data_urls = [base_url + s for s in app_ids]
    
    return data_urls
            

In [ ]:
with open("config.json") as json_data_file:
    data = json.load(json_data_file)

def get_collection_or_category_urls(collection_or_category_name, collection_or_category, geographic_location):
    url = 'https://play.google.com/store/apps/' + collection_or_category +'/' + collection_or_category_name + '?gl=' + geographic_location
    http = httplib2.Http()
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--incognito")
    if platform.system() == 'Windows':
        driver = webdriver.Chrome(executable_path='./chromedriver.exe')
    else:
        if platform.system() == 'Darwin':
            driver = webdriver.Chrome()
    driver.implicitly_wait(30)
    # provide url(s) for driver
    driver.get(url)
    # scroll to get all apps
    scroll(driver, 2)

    # read and obtain all urls
    if platform.system() == 'Windows':
        soup_a = BeautifulSoup(driver.page_source, features='html.parser')
    else:
        if platform.system() == 'Darwin':
            soup_a = BeautifulSoup(driver.page_source, 'lxml')
    
    driver.close()

    # copy all urls
    urls = []
    for link in soup_a.find_all('a'):
        urls.append(link.get('href'))
    app_urls = [val for val in urls if val.startswith('/store/apps/details?id=')]
    # print("No. of apps found for " + collection_or_category + " " + collection_or_category_name + ":", len(app_urls))    
    # print(app_urls)
    return app_urls
all_urls = []
geo_locations = []
for location in data['gl']:
    geo_locations.append(data['gl'][location])
for gl_code in geo_locations:
    for config_key in data['category']:
        all_urls = all_urls + get_collection_or_category_urls(data['category'][config_key],'category',gl_code)
    for config_key in data['collection']:
        all_urls = all_urls + get_collection_or_category_urls(data['collection'][config_key],'collection',gl_code) 
seen = set()
total_unique_urls = []
for item in all_urls:
    if item not in seen:
        seen.add(item)
        total_unique_urls.append(item)     
print("Total number of apps found: ", len(total_unique_urls))
print(total_unique_urls)

In [ ]:
# remove extra characters from urls array to get only app ids
for i in range(len(total_unique_urls)):
    total_unique_urls[i] = total_unique_urls[i][23:]
    print(total_unique_urls[i])
print(len(total_unique_urls))
print("------------------------------------------------------------------------------------------------------------")
data_urls = get_data_safety_urls(total_unique_urls)
print(len(data_urls))
print(data_urls, total_unique_urls)

In [ ]:
# pd.set_option('max_colwidth', 400)
# df = pd.DataFrame(data_urls, columns=['data safety urls'])
# display(df)

In [ ]:
# appid_df = pd.DataFrame(app_ids, columns=['app id'])
# display(appid_df)

In [ ]:
# concatenate app id and data safety urls dfs
df = pd.DataFrame(data_urls, columns=['data safety urls'])
appid_df = pd.DataFrame(total_unique_urls, columns=['app id'])
pd.set_option('max_colwidth', 400)
all_df = pd.concat([appid_df, df], axis=1, ignore_index=True)

all_df.columns = ['app id', 'data safety url']
display(all_df)

In [ ]:
darr = all_df['data safety url'].to_numpy()
print(darr)

In [ ]:
# press privacy policy button on page
if platform.system() == 'Windows':
        driver = webdriver.Chrome(executable_path='./chromedriver.exe')
else:
    if platform.system() == 'Darwin':
        driver = webdriver.Chrome()
driver.implicitly_wait(2)
temp = []
df = pd.DataFrame(columns=['privacy_policy_url'])
for i in darr:
#     temp += press_button(driver, i)
    df = df.append({'privacy_policy_url':press_button(driver,i)}, ignore_index=True)
#     urls = urls.append({'privacy_policy_url': str(priv_url)}, ignore_index=True)
#     print(i)
# urls = url.concat(urls, df)
# df.insert(loc=0, column = 'privacy_policy_url', value=temp)
driver.close()
# display(urls)

In [ ]:
display(df)

In [ ]:
display(all_df)

In [ ]:
print(len(df.index))
print(len(all_df.index))


merged = pd.concat([df, all_df], axis=1)
display(merged)

In [ ]:
# driver = webdriver.Chrome()
# driver.implicitly_wait(2)
# temp = press_button(driver, "https://play.google.com/store/apps/datasafety?id=com.apple.atve.androidtv.appletv")
# df.insert(loc=1, column = 'privacy policy url', value=temp)
# driver.close()

In [ ]:
link = "https://www.apple.com/legal/privacy/pdfs/apple-privacy-policy-en-ww.pdf"
policy = []

r = requests.get(link)
f = io.BytesIO(r.content)

reader = PdfFileReader(f)
count = reader.numPages

for i in range(count):
    policy += reader.getPage(i).extractText().split('\n')

In [ ]:
print(policy)

In [ ]:
# begin working sample of reading privacy policy html page with no dropdown menu's

# url = "https://snap.com/en-US/privacy/privacy-policy"
url = "https://www.whatsapp.com/legal/privacy-policy"

res = requests.get(url)
html_page = res.content

soup = BeautifulSoup(html_page, 'html.parser')
text = soup.find_all(text=True)

In [ ]:
set([t.parent.name for t in text])

In [ ]:
output = ''
blacklist = [
    '[document]',
   'noscript',
    'header',
    'html',
    'meta',
    'head', 
    'input',
    'script',
    'link',
    'style'
]

for t in text:
    if t.parent.name not in blacklist:
        output += '{} '.format(t)

In [ ]:
print(output)

# end html page read